In [1]:
import json
import numpy as np
import pandas as pd

In [2]:
from statsmodels.tsa.api import ExponentialSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX
from catboost import CatBoostRegressor

In [3]:
from supporting_forecast_fuctions import *

In [4]:
argv = [123, 'GOOG' , 12, 'SARIMA']

In [6]:
user_id = argv[0]
ticker = argv[1]
horizon  = argv[2]
name_model  = argv[3]

'''with open(str(user_id) + '_add_params.json') as json_file:
    add_params = json.load(json_file)

with open(str(user_id) + '_kwargs_model.json') as json_file:
    kwargs_model  = json.load(json_file)

with open(str(user_id) + '_add_data.json') as json_file:
    add_data = pd.read_csv(json_file)'''

#kwargs_model = {'trend' : 'mul', 'damped_trend': True, 'initialization_method': 'estimated'}
kwargs_model = {'order' : (1, 1, 1), 'enforce_invertibility': True, 'enforce_stationarity': True}
#kwargs_model = {'maxlags': 4}
yf_data = get_yfin_data(ticker)




if name_model == 'EXP_SM':

    train_main_series = yf_data['Close']

    exp_sm = ExponentialSmoothing(
        train_main_series,
        **kwargs_model
    ).fit()

    forecast_numpy = exp_sm.forecast(horizon).values
    forecast_series = pd.Series(data = forecast_numpy, index=pd.date_range(train_main_series.index[-1], periods=horizon))
    metrics_df = evaluate_exp_sm(train_main_series, kwargs_model)
    fig_forecast  = plot_prediction(train_main_series,forecast_series)
    
elif name_model == 'VAR':
    train_main_series  = yf_data['Close']

    VAR_model = VAR(yf_data)
    VAR_model_res = VAR_model.fit(**kwargs_model)
    forecast_numpy = VAR_model_res.forecast(yf_data.values, horizon)
    forecast_df = pd.DataFrame(data = forecast_numpy, index=pd.date_range(yf_data.index[-1], periods=horizon), columns = yf_data.columns)
    forecast_series = forecast_df['Close']
    metrics_df = evaluate_VAR(yf_data, kwargs_model=kwargs_model)
    fig_forecast = plot_prediction(train_main_series,forecast_series)
elif name_model == 'SARIMA':

    train_main_series = yf_data

    model = SARIMAX(train_main_series['Close'], **kwargs_model).fit()
    predict_numpy = model.forecast(steps = horizon).values

    metrics_df = evaluate_sarima(train_main_series, kwargs_model)
    forecast_series = pd.Series(data = predict_numpy, index=pd.date_range(train_main_series.index[-1], periods=horizon))
    fig_forecast = plot_prediction(train_main_series['Close'],forecast_series)
    #fig.write_image("fig1.png")


elif name_model == 'catboost':
    train_main_series  = yf_data
    train_main_series = add_MA(train_main_series)
    train_main_series = add_lags(train_main_series, horizon+1)
    train_main_series = pd.concat([train_main_series, encode_time(train_main_series.index)], axis=1)
    x_train = train_main_series.iloc[:,-51:-3]
    y_train = train_main_series['Close']

    model = CatBoostRegressor(iterations= 500, loss_function= 'MAE')
    model.fit(x_train, y_train)
    
    x_pred = train_main_series.iloc[-horizon:,:48]
    x_pred.columns = train_main_series.iloc[-horizon:,-51:-3].columns

    y_pred = model.predict(x_pred)
    diff = y_pred[0] - y_train[-1]
    y_pred = y_pred - diff
    forecast_series = pd.Series(data = y_pred, index=pd.date_range(train_main_series.index[-1], periods=horizon))
    fig_forecast = plot_prediction(train_main_series['Close'],forecast_series)
    metrics_df = evaluate_catboost(train_main_series)
    

fig_forecast.write_image(str(user_id) + '_forecast.png', engine="orca")



[*********************100%%**********************]  1 of 1 completed


c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\base\tsa_model.py:834: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
c:\Users\kir12\.conda\envs\vega_time_series\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: A date index has been provided, but it has no associated frequency information and so

NameError: name 'fig_forecast' is not defined

In [ ]:
metrics_df

,60,30,14,7
MAE,3.708685,2.693242,2.288271,1.758812
MAPE,0.027897,0.020021,0.016834,0.012997
RMSE,4.520325,3.167995,2.771628,2.242925
